In [ ]:
import catalyst
from catalyst import qjit

import pennylane as qml
from pennylane import numpy as np
import matplotlib.pyplot as plt

from scipy.optimize import minimize, OptimizeResult

import jax.numpy as jnp

In [ ]:
## Layers and wires for circuits

symbols = ["Li", "H"]
coordinates = np.array([0.0, 0.0, 0.0, 0.0, 0.0, 1.0])

# Building the molecular hamiltonian for LiH
hamiltonian, qubits = qml.qchem.molecular_hamiltonian(
    symbols,
    coordinates,
    method="pyscf",
)

LAYERS = 1
WIRES = qubits

# Encoder: 
# Linear -> 1
# Gaussian -> 2

ENCODER_MULTIPLIER_DICT = {1:2,
                      2:4}

ENCODER = 1

shared_dev = qml.device("lightning.qubit", wires=WIRES)

symbols = ["Li", "H"]

train_points = [1.0,2.0]

test_points = np.arange(0.6,6,0.2)


In [ ]:
def linear_encoding(param_array, r):
    """1-D array with alphas and betas. len(param_array) = 2 * len(weights) 

    Args:
        param_array (float): alphas and betas for lineasr encoding
        r(float): Hamiltonian parameter (in this case, distance)
    """
    return param_array[::2]*r + param_array[1::2]

In [ ]:
def gaussian_encoding(param_array, r):
    """1-D array with alphas, betas, gammas and deltas. len(param_array) = 4 * len(weights) 

    Args:
        param_array (float): , betas, gammas and deltas for gaussian encoding
        r(float): Hamiltonian parameter (in this case, distance)
    """

    exp_arg = param_array[1::4]*[i-r for i in param_array[2::4]]

    return param_array[::4]*np.exp(exp_arg) + param_array[3::4]

In [ ]:
## Get shapes

## For linear is 2
## For Gaussian is 4
ENCODING_MULTIPLIER= ENCODER_MULTIPLIER_DICT[ENCODER]

if(ENCODER == 1):
    ENCODER_FUNC =linear_encoding
else:
    ENCODER_FUNC =gaussian_encoding

num_params = (WIRES + LAYERS*(WIRES-1)*2)*ENCODING_MULTIPLIER

shapes = qml.SimplifiedTwoDesign.shape(n_layers=LAYERS, n_wires=WIRES)
weights = np.random.random(num_params)

print("===== NUM OF PARAMETERS =====")
print(num_params)

In [ ]:
##Run for all points without training

@qml.qnode(shared_dev)
def catalyst_simplified_two_design(params, d):

    coordinates = jnp.array([0.0, 0.0, 0.0, 0.0, 0.0, d])

    # Building the molecular hamiltonian for LiH
    hamiltonian, qubits = qml.qchem.molecular_hamiltonian(
        symbols,
        coordinates,
        method="pyscf",
    )    

    shapes = qml.SimplifiedTwoDesign.shape(n_layers=LAYERS, n_wires=qubits)

    init_weights, weights = np.reshape(params[:qubits],shapes[0]), np.reshape(params[qubits:],shapes[1])

    qml.SimplifiedTwoDesign(initial_layer_weights=init_weights, weights=weights, wires=range(qubits))

    return qml.expval(
        qml.Hamiltonian(np.array(hamiltonian.coeffs), hamiltonian.ops)
    )  

In [ ]:
from jax.core import ShapedArray

@qjit
def energy_simplified_two_design(params: ShapedArray(shape=(num_params,), dtype=jnp.float64)):

    energies = []   

    for r in train_points:
   
        def catalyst_simplified_two_design(params):
        
            symbols = ["Li", "H"]
            coordinates = np.array([0.0, 0.0, 0.0, 0.0, 0.0, r])     
            
            # Building the molecular hamiltonian for LiH
            hamiltonian, qubits = qml.qchem.molecular_hamiltonian(
                symbols,
                coordinates,
                method="pyscf",
            )
            
            print(coordinates)           
            
            shapes = qml.SimplifiedTwoDesign.shape(n_layers=LAYERS, n_wires=qubits)   

            weights_encoded =  ENCODER_FUNC(params, r)       

            init_weights, weights = np.reshape(weights_encoded[:qubits],shapes[0]), np.reshape(weights_encoded[qubits:],shapes[1])

            qml.SimplifiedTwoDesign(initial_layer_weights=init_weights, weights=weights, wires=range(qubits))

            return qml.expval(
                qml.Hamiltonian(np.array(hamiltonian.coeffs), hamiltonian.ops)
            )  
                
        circuit = qml.QNode(catalyst_simplified_two_design, shared_dev)
        
        energies.append(circuit(params))
        
    join_energy = jnp.sum(jnp.array(energies))


    
    return join_energy/len(train_points) 

In [ ]:
from jax.core import ShapedArray

@qjit
def gradient_simplified_two_design(params: ShapedArray(shape=(num_params,), dtype=jnp.float64)):

    gradients = []

    for r in train_points:
   
        def catalyst_simplified_two_design(params):
        
            symbols = ["Li", "H"]
            coordinates = np.array([0.0, 0.0, 0.0, 0.0, 0.0, r])     
            
            # Building the molecular hamiltonian for LiH
            hamiltonian, qubits = qml.qchem.molecular_hamiltonian(
                symbols,
                coordinates,
                method="pyscf",
            )
            
            print(coordinates)           
            
            shapes = qml.SimplifiedTwoDesign.shape(n_layers=LAYERS, n_wires=qubits)   

            weights_encoded =  ENCODER_FUNC(params, r)       

            init_weights, weights = np.reshape(weights_encoded[:qubits],shapes[0]), np.reshape(weights_encoded[qubits:],shapes[1])

            qml.SimplifiedTwoDesign(initial_layer_weights=init_weights, weights=weights, wires=range(qubits))

            return qml.expval(
                qml.Hamiltonian(np.array(hamiltonian.coeffs), hamiltonian.ops)
            )  
                
        circuit = qml.QNode(catalyst_simplified_two_design, shared_dev)
        
        gradients.append(catalyst.grad(circuit)(params))
        
    all_gradients = jnp.array(gradients)
    joined_gradients = jnp.sum(jnp.array([all_gradients[i][0] for i in range(len(all_gradients))]), axis=0)

    
    return joined_gradients/len(train_points)

In [ ]:
energy_simplified_two_design(weights)


In [ ]:
gradient_simplified_two_design(weights)

In [ ]:
def gradient_descent(fun,x0, stepsize=0.1, tol=1e-4,maxiter=100, **options):

    new_params = jnp.array(x0)
    ref_energy = energy_simplified_two_design(new_params)

    niter = 0

    for i in range(maxiter):
        niter +=1
        grad = gradient_simplified_two_design(new_params)

        new_params -= grad*stepsize

        new_energy = energy_simplified_two_design(new_params)

        print("Step: ",i, " Energy: ", new_energy)

        if(np.abs(new_energy-ref_energy) < tol):
            break
        else:
            ref_energy = new_energy

    return OptimizeResult(x=new_params, nit=niter)


In [ ]:
def spsa_optimizer(fun, x0, maxiter=100,alpha=0.602,gamma=0.101, c=0.2, A=None, a=None, tol=1e-4, **options):
    new_params = jnp.array(x0)
    ref_energy = energy_simplified_two_design(new_params)

    if not A:
        A = maxiter * 0.1

    if not a:
        a = 0.05 * (A + 1) ** alpha        

    niter = 0

    for i in range(maxiter):
        niter +=1

        ak=a/np.power(i+1+A,alpha)
        ck=c/np.power(i+1,gamma)

        delta = np.random.choice([-1, 1], size=x0.shape)

        thetaplus=new_params+ck*delta
        thetaminus=new_params-ck*delta
        yplus=energy_simplified_two_design(thetaplus)
        yminus=energy_simplified_two_design(thetaminus)  

        grad = jnp.array([(yplus - yminus) / (2 * ck * di) for di in delta]  )

        new_params -= ak*grad

        new_energy = energy_simplified_two_design(new_params)

        print("Step: ",i, " Energy: ", new_energy)

        if(np.abs(new_energy-ref_energy) < tol):
            break
        else:
            ref_energy = new_energy

    return OptimizeResult(x=new_params, nit=niter)    

In [ ]:
def adam(fun, x0, maxiter=100,stepsize=0.01, beta1=0.9, beta2=0.99, tol=1e-4,eps=1e-08, **options):
    new_params = jnp.array(x0)
    ref_energy = energy_simplified_two_design(new_params)
      
    m = jnp.zeros_like(x0)
    v = jnp.zeros_like(x0)

    niter = 0

    for i in range(maxiter):
        niter +=1

        grad = gradient_simplified_two_design(new_params)

        m = beta1*m + (1-beta1)*grad
        v = beta2*v + (1-beta2)*jnp.square(grad)

        mhat = m/(1-beta1**(i+1))

        vhat = v/(1-beta2**(i+1))

        step = mhat/(jnp.array([np.sqrt(vhat_i) + eps for vhat_i in vhat]) )


        new_params -= stepsize*step

        new_energy = energy_simplified_two_design(new_params)

        print("Step: ",i, " Energy: ", new_energy)

        if(np.abs(new_energy-ref_energy) < tol):
            break
        else:
            ref_energy = new_energy

    return OptimizeResult(x=new_params, nit=niter)    

In [ ]:
res = minimize(energy_simplified_two_design, weights, method = adam, tol=1e-6)

In [ ]:
res.x

In [ ]:
res = minimize(energy_simplified_two_design, weights, method = 'COBYLA', tol=1e-4)

In [ ]:
res.x

In [ ]:
energy_simplified_two_design(res.x)

In [ ]:
trained_energies = []

for d in test_points:
    print(d)
    encoded_params = linear_encoding(res.x, d)
    energy= catalyst_simplified_two_design(encoded_params,d)
    trained_energies.append(energy)

In [ ]:
plt.plot(test_points, trained_energies, label="trained", marker='o')
plt.legend()
plt.title('Trained-Lineal Encoding')
plt.xlabel('Distance')
plt.ylabel('Energy(Eh)')
plt.show

In [ ]:
jnp.sqrt(np.array([1,2,3]))